In [11]:
import itertools
import numpy as np
import pandas as pd
import tensorly as tl
from tensorly.cp_tensor import cp_to_tensor
import matplotlib.pyplot as plt
np.random.seed(42)

I, J, K_tensor = 8, 8, 8
rank = 3
tensor = tl.tensor(np.random.uniform(0, 1, (I, J, K_tensor)))

alpha_0 = 1e3
alpha_final = 1e15

dt_list = [0.01]
nu_list = [5000]
lambda_sigma_list = [(1.5, 8.5)]
K_list = [1000]
eta_list = [0.85]

param_grid = list(itertools.product(dt_list, nu_list, lambda_sigma_list, K_list, eta_list))

def alpha_sequence(alpha_0, alpha_K, K):
    ks = np.arange(K+1)
    alphas = alpha_0 + (ks / K) * (alpha_K - alpha_0)
    return alphas

def objective_function(particle, tensor, rank):
    A = particle['A']
    B = particle['B']
    C = particle['C']
    reconstructed_tensor = cp_to_tensor((np.ones(rank), [A, B, C]))
    error = tl.norm(tensor - reconstructed_tensor)
    return error

def compute_consensus_point(particles, alpha, tensor, rank):
    min_energy_particle = min(particles, key=lambda p: objective_function(p, tensor, rank))
    min_energy = objective_function(min_energy_particle, tensor, rank)

    numerator_A = np.zeros_like(particles[0]['A'])
    numerator_B = np.zeros_like(particles[0]['B'])
    numerator_C = np.zeros_like(particles[0]['C'])
    denominator = 0.0

    for particle in particles:
        energy = objective_function(particle, tensor, rank)
        weight = np.exp(-alpha * (energy - min_energy))
        numerator_A += weight * particle['A']
        numerator_B += weight * particle['B']
        numerator_C += weight * particle['C']
        denominator += weight

    consensus_A = numerator_A / denominator
    consensus_B = numerator_B / denominator
    consensus_C = numerator_C / denominator
    return {'A': consensus_A, 'B': consensus_B, 'C': consensus_C}

def project_matrices_to_ball(particles, consensus_point, eta):
    for particle in particles:
        for key in ['A', 'B', 'C']:
            vec_particle = particle[key].flatten()
            vec_consensus = consensus_point[key].flatten()
            distance = np.linalg.norm(vec_particle - vec_consensus)
            radius = eta * distance
            if distance > radius:
                direction = (vec_particle - vec_consensus) / distance
                vec_particle = vec_consensus + radius * direction
            particle[key] = vec_particle.reshape(particle[key].shape)
    return particles

def anisotropic_update(particles, consensus_point, lambda_, sigma, dt, tensor, rank):
    consensus_point_loss = objective_function(consensus_point, tensor, rank)
    for particle in particles:
        A, B, C = particle['A'], particle['B'], particle['C']
        A_consensus, B_consensus, C_consensus = consensus_point['A'], consensus_point['B'], consensus_point['C']

        current_loss = objective_function(particle, tensor, rank)
        if consensus_point_loss < current_loss:
            drift_A = (-lambda_) * (A - A_consensus) * dt
            drift_B = (-lambda_) * (B - B_consensus) * dt
            drift_C = (-lambda_) * (C - C_consensus) * dt
        else:
            drift_A = np.zeros_like(A)
            drift_B = np.zeros_like(B)
            drift_C = np.zeros_like(C)

        # Normal distribution noise
        B_A = np.random.normal(loc=0, scale=1, size=A.shape)
        B_B = np.random.normal(loc=0, scale=1, size=B.shape)
        B_C = np.random.normal(loc=0, scale=1, size=C.shape)

        diffusion_A = sigma * (A - A_consensus) * B_A * np.sqrt(dt)
        diffusion_B = sigma * (B - B_consensus) * B_B * np.sqrt(dt)
        diffusion_C = sigma * (C - C_consensus) * B_C * np.sqrt(dt)

        particle['A'] += drift_A + diffusion_A
        particle['B'] += drift_B + diffusion_B
        particle['C'] += drift_C + diffusion_C

    return particles

def resample_particles(particles, consensus_point, noise_scale=0.01):
    """
    Resample each particle's matrices A, B, C around the consensus point.
    noise_scale controls the standard deviation of the Gaussian noise.
    """
    for particle in particles:
        for key in ['A', 'B', 'C']:
            shape = consensus_point[key].shape
            # Replace the particle matrix with consensus + small noise
            noise = np.random.normal(loc=0.0, scale=noise_scale, size=shape)
            noise = np.clip(noise, -0.05, 0.05)

            particle[key] = consensus_point[key] + noise
    return particles

results = []

for dt, nu, (lambda_, sigma), K, eta in param_grid:
    print(f"Running with dt={dt}, nu={nu}, lambda={lambda_}, sigma={sigma}, K={K}, eta={eta}")

    particles = []
    for _ in range(nu):
        A = np.random.uniform(0, 1, (I, rank))
        B = np.random.uniform(0, 1, (J, rank))
        C = np.random.uniform(0, 1, (K_tensor, rank))
        particles.append({'A': A, 'B': B, 'C': C})

    alpha_values = alpha_sequence(alpha_0, alpha_final, K)

    for iteration in range(K + 1):
        alpha_current = alpha_values[iteration]

        # Compute consensus point
        consensus_point = compute_consensus_point(particles, alpha_current, tensor, rank)
        consensus_tensor = cp_to_tensor((np.ones(rank),
                                         [consensus_point['A'],
                                          consensus_point['B'],
                                          consensus_point['C']]))

        consensus_abs_error = tl.norm(tensor - consensus_tensor)
        nonzero_mask = tensor != 0
        consensus_rel_error = (
            tl.norm(tensor[nonzero_mask] - consensus_tensor[nonzero_mask]) /
            tl.norm(tensor[nonzero_mask])
        )

        if iteration < K:
            # 1) Apply drift-diffusion
            particles = anisotropic_update(
                particles, consensus_point,
                lambda_=lambda_, sigma=sigma,
                dt=dt, tensor=tensor, rank=rank
            )
            # 2) (Optional) Project to the shrinking ball
            particles = project_matrices_to_ball(particles, consensus_point, eta)
            # 3) Resample around consensus to maintain diversity
            particles = resample_particles(particles, consensus_point, noise_scale=0.01)


    results.append({
        'dt': dt,
        'nu': nu,
        'lambda': lambda_,
        'sigma': sigma,
        'eta': eta,
        'K': K,
        'consensus_abs_error': consensus_abs_error,
        'consensus_rel_error': consensus_rel_error
    })

df_results_with_heurs = pd.DataFrame(results)
df_results_with_heurs


Running with dt=0.01, nu=5000, lambda=1.5, sigma=8.5, K=1000, eta=0.85


,dt,nu,lambda,sigma,eta,K,consensus_abs_error,consensus_rel_error
0,0.01,5000,1.5,8.5,0.85,1000,6.024777,0.456325
